# CNN Architectures 

## COVIDx Dataset

In [1]:
from fastai.vision.all import *
path = Path('/home/jupyter/covidx') 

In [2]:
torch.cuda.empty_cache()

In [3]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
SEED = 42
seed_everything(SEED)

In [4]:
df = pd.read_csv(path/'covidx.csv')
df

,patientid,path,finding,source,is_valid
0,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg,pneumonia,cohen,False
1,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg,pneumonia,cohen,False
2,7,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg,pneumonia,cohen,False
3,7,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg,pneumonia,cohen,False
4,9,SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day5.jpeg,pneumonia,cohen,False
...,...,...,...,...,...
15491,2c917d3a-95cb-4c11-802c-f83e28cb37bc,2c917d3a-95cb-4c11-802c-f83e28cb37bc.png,pneumonia,rsna,True
15492,3040d9d7-d895-453f-887c-616c10531960,3040d9d7-d895-453f-887c-616c10531960.png,pneumonia,rsna,True
15493,c07f52df-d481-434f-84c1-04263926ac40,c07f52df-d481-434f-84c1-04263926ac40.png,pneumonia,rsna,True
15494,c109061a-d815-4cae-8343-9230d8024adf,c109061a-d815-4cae-8343-9230d8024adf.png,pneumonia,rsna,True


In [5]:
get_x=lambda x:path/"images"/f"{x[1]}"
get_y=lambda x:x[2]
splitter=ColSplitter('is_valid')
bs = 32

In [6]:
metrics=[accuracy,  
         #RocAuc(average='macro', multi_class='ovr'),
         MatthewsCorrCoef(sample_weight=None),
         Precision(average='macro'),
         Recall(average='macro'),     
         F1Score(average='macro')] 

In [7]:
item_tfms=Resize(480, method='squish', pad_mode='zeros', resamples=(2, 0))
batch_tfms=[*aug_transforms(mult=1.0, do_flip=False, flip_vert=False, 
                            max_rotate=20.0, max_zoom=1.2, max_lighting=0.3, max_warp=0.2, 
                            p_affine=0.75, p_lighting=0.75, 
                            xtra_tfms=None, size=None, mode='bilinear', pad_mode='reflection', 
                            align_corners=True, batch=False, min_scale=1.0),
                            Normalize.from_stats(*imagenet_stats)]

In [8]:
db = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
               get_x=get_x,
               get_y=get_y,
               splitter=splitter,
               item_tfms = item_tfms,
               batch_tfms=batch_tfms)

In [9]:
dl = db.dataloaders(df, bs=bs)

## Vgg16

In [10]:
from torchvision.models import vgg16
arch = vgg16

In [11]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [12]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.512827,0.446783,0.839772,0.935025,0.700845,0.823890,0.719592,0.756066,03:13


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.342844,0.303075,0.891070,0.959938,0.794860,0.887265,0.770591,0.810977,08:55
1,0.313129,0.273016,0.904370,0.966267,0.819775,0.903046,0.777016,0.818010,08:55
2,0.252762,0.274075,0.901837,0.962911,0.815361,0.871917,0.776432,0.809726,08:55
3,0.249662,0.259520,0.906270,0.973798,0.823395,0.921487,0.784797,0.828663,08:55
4,0.239050,0.272781,0.911336,0.974381,0.833417,0.923242,0.784480,0.827001,08:55
5,0.215626,0.276722,0.910703,0.972510,0.831722,0.922127,0.765072,0.808734,08:55
6,0.208460,0.245813,0.917036,0.980276,0.843996,0.930153,0.804503,0.847220,08:55
7,0.179624,0.217267,0.921469,0.981628,0.853172,0.919524,0.832638,0.865912,08:55
8,0.188366,0.214870,0.924003,0.985775,0.858857,0.938720,0.836728,0.873536,08:56
9,0.170084,0.274817,0.924003,0.979906,0.857756,0.943333,0.804398,0.850800,08:55


## Vgg19

In [11]:
from torchvision.models import vgg19
arch = vgg19

In [12]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [13]:
learn.summary()

Sequential (Input shape: ['32 x 3 x 480 x 480'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               32 x 64 x 480 x 480  1,792      False     
________________________________________________________________
ReLU                 32 x 64 x 480 x 480  0          False     
________________________________________________________________
Conv2d               32 x 64 x 480 x 480  36,928     False     
________________________________________________________________
ReLU                 32 x 64 x 480 x 480  0          False     
________________________________________________________________
MaxPool2d            32 x 64 x 240 x 240  0          False     
________________________________________________________________
Conv2d               32 x 128 x 240 x 24  73,856     False     
________________________________________________________________
ReLU                 32 x 128 x 240 x 24  0          False     
_________________________________________________

In [14]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.522969,0.452123,0.843572,0.934193,0.704788,0.871197,0.672691,0.717129,04:00


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.351209,0.358075,0.872704,0.951519,0.757987,0.899085,0.688339,0.725815,11:05
1,0.294502,0.298938,0.894870,0.964146,0.801339,0.924930,0.752888,0.799188,11:05
2,0.274115,0.286902,0.906270,0.965639,0.823353,0.921000,0.781287,0.825471,11:04
3,0.250115,0.266896,0.915136,0.971298,0.840331,0.922770,0.793765,0.836428,11:04
4,0.222488,0.256176,0.915769,0.972928,0.842912,0.925296,0.789515,0.830266,11:04
5,0.237932,0.270503,0.913236,0.978271,0.837696,0.936271,0.781357,0.825119,11:04
6,0.187826,0.261877,0.923369,0.977766,0.857152,0.943235,0.812329,0.854493,11:04
7,0.198189,0.229433,0.924003,0.981021,0.858017,0.924536,0.826382,0.862170,11:05
8,0.183869,0.255114,0.922736,0.978718,0.855910,0.925794,0.828225,0.866120,11:04
9,0.150950,0.264671,0.923369,0.982888,0.857165,0.943298,0.809557,0.851958,11:04


## ResNet18

In [10]:
arch = resnet18

In [11]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [12]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.481998,0.361954,0.867638,0.949686,0.749649,0.889521,0.728758,0.775386,02:21


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.293086,0.258352,0.911970,0.972329,0.835456,0.935450,0.783745,0.827312,01:35
1,0.252105,0.237397,0.914503,0.979504,0.839344,0.932018,0.792277,0.835995,01:35
2,0.212356,0.246180,0.916403,0.980187,0.842705,0.939728,0.772525,0.818045,01:35
3,0.215215,0.241520,0.924003,0.979766,0.857354,0.939324,0.813079,0.855758,01:35
4,0.215960,0.250787,0.915769,0.982363,0.841964,0.938429,0.776397,0.820827,01:35
5,0.185635,0.212851,0.924636,0.981131,0.858497,0.946858,0.824175,0.867717,01:35
6,0.173318,0.178058,0.931602,0.988620,0.872076,0.934819,0.865090,0.894266,01:35
7,0.160086,0.179484,0.939835,0.986603,0.887811,0.951047,0.880337,0.910515,01:35
8,0.171640,0.175663,0.934136,0.989612,0.877986,0.939303,0.884148,0.907551,01:35
9,0.155360,0.219246,0.930969,0.982403,0.870702,0.945346,0.847343,0.884836,01:35


## ResNet34

In [22]:
arch = resnet34

In [23]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [24]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.501349,0.392193,0.859405,0.942163,0.734192,0.892093,0.718875,0.767483,01:51


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.294381,0.264364,0.909436,0.971731,0.829431,0.935781,0.792963,0.838921,02:38
1,0.250155,0.238602,0.914503,0.976433,0.839053,0.933486,0.787475,0.833021,02:38
2,0.215566,0.265954,0.910070,0.980737,0.830965,0.934660,0.754714,0.797937,02:38
3,0.205807,0.281876,0.915136,0.980310,0.841159,0.944114,0.788410,0.837606,02:38
4,0.210056,0.244742,0.921469,0.983018,0.853480,0.942147,0.805286,0.848064,02:37
5,0.178306,0.212665,0.929069,0.983631,0.867242,0.948577,0.821632,0.864240,02:38
6,0.177272,0.178505,0.936035,0.988245,0.880441,0.942869,0.877150,0.905166,02:38
7,0.151102,0.168611,0.946168,0.986342,0.900184,0.933056,0.917365,0.924806,02:38
8,0.179091,0.146250,0.948068,0.991054,0.903738,0.937643,0.927361,0.932375,02:38
9,0.148980,0.196761,0.934769,0.985673,0.877896,0.950094,0.860691,0.896541,02:38


## ResNet50

In [10]:
arch = resnet50

In [11]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [12]:
learn.summary()

Sequential (Input shape: ['32 x 3 x 480 x 480'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               32 x 64 x 240 x 240  9,408      False     
________________________________________________________________
BatchNorm2d          32 x 64 x 240 x 240  128        True      
________________________________________________________________
ReLU                 32 x 64 x 240 x 240  0          False     
________________________________________________________________
MaxPool2d            32 x 64 x 120 x 120  0          False     
________________________________________________________________
Conv2d               32 x 64 x 120 x 120  4,096      False     
________________________________________________________________
BatchNorm2d          32 x 64 x 120 x 120  128        True      
________________________________________________________________
Conv2d               32 x 64 x 120 x 120  36,864     False     
_________________________________________________

In [13]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.442210,0.403942,0.874604,0.933376,0.763263,0.897877,0.694451,0.728305,02:55


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.263174,0.271290,0.899937,0.970570,0.811286,0.928027,0.749430,0.793860,03:44
1,0.221137,0.253178,0.913870,0.979552,0.839221,0.923510,0.782288,0.823245,03:43
2,0.210977,0.210161,0.928436,0.979961,0.865828,0.948533,0.842326,0.882166,03:43
3,0.217748,0.292778,0.908803,0.977539,0.828099,0.936663,0.747498,0.793358,03:43
4,0.184755,0.172017,0.935402,0.987911,0.880565,0.932180,0.876769,0.899928,03:43
5,0.199449,0.169885,0.935402,0.990332,0.882003,0.893085,0.936092,0.912384,03:43
6,0.164780,0.211289,0.934769,0.981047,0.879438,0.922435,0.892296,0.905997,03:43
7,0.177293,0.151098,0.941102,0.991495,0.891088,0.952155,0.909542,0.928363,03:44
8,0.169335,0.157909,0.944902,0.989567,0.897125,0.953288,0.892032,0.918598,03:43
9,0.148238,0.209123,0.934769,0.985528,0.879121,0.940364,0.895622,0.915810,03:43


## Efficientnet-B0

In [10]:
from efficientnet_pytorch import EfficientNet
arch = EfficientNet.from_pretrained("efficientnet-b0")

Loaded pretrained weights for efficientnet-b0


In [11]:
learn = Learner(dl, model=arch, metrics=metrics)

In [12]:
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.332779,0.613781,0.860038,0.733682,0.901387,0.642549,0.656213,02:50


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.196911,0.200902,0.929702,0.868577,0.942839,0.819052,0.860416,02:34
1,0.167607,0.154858,0.945535,0.898501,0.945457,0.891112,0.914731,02:34
2,0.164230,0.357306,0.918303,0.846589,0.940980,0.743346,0.783424,02:34
3,0.178583,0.159113,0.951235,0.909251,0.939550,0.900415,0.918104,02:33
4,0.178495,0.235590,0.920836,0.852929,0.949509,0.811574,0.859620,02:34
5,0.145955,0.227957,0.932869,0.874313,0.950926,0.803195,0.847757,02:33
6,0.160827,0.362794,0.909436,0.830508,0.934239,0.721255,0.754636,02:33
7,0.149919,0.302451,0.886637,0.804177,0.784058,0.887333,0.808373,02:33
8,0.165564,0.229959,0.922736,0.858695,0.926200,0.817677,0.853069,02:34
9,0.162324,0.262356,0.925269,0.862473,0.955189,0.854866,0.895007,02:33
